<H1>Financial Distress Prediction
    
<H4>https://www.kaggle.com/boardgamefreak/financial-distress-prediction-forward-chaining

Загрузка необходимых библиотек

In [1]:
import numpy as np 
import pandas as pd
import os
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score

Загрузка датафрейма

In [2]:
df = pd.read_csv('Financial Distress.csv', index_col=False,
                 dtype={
                     'Company': np.uint16,
                     'Time': np.uint8,
                     'Financial Distress': np.double
                 })

In [3]:
df.head()

,Company,Time,Financial Distress,x1,x2,x3,x4,x5,x6,x7,...,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83
0,1,1,0.010636,1.2810,0.022934,0.87454,1.21640,0.060940,0.188270,0.52510,...,85.437,27.07,26.102,16.000,16.0,0.2,22,0.060390,30,49
1,1,2,-0.455970,1.2700,0.006454,0.82067,1.00490,-0.014080,0.181040,0.62288,...,107.090,31.31,30.194,17.000,16.0,0.4,22,0.010636,31,50
2,1,3,-0.325390,1.0529,-0.059379,0.92242,0.72926,0.020476,0.044865,0.43292,...,120.870,36.07,35.273,17.000,15.0,-0.2,22,-0.455970,32,51
3,1,4,-0.566570,1.1131,-0.015229,0.85888,0.80974,0.076037,0.091033,0.67546,...,54.806,39.80,38.377,17.167,16.0,5.6,22,-0.325390,33,52
4,2,1,1.357300,1.0623,0.107020,0.81460,0.83593,0.199960,0.047800,0.74200,...,85.437,27.07,26.102,16.000,16.0,0.2,29,1.251000,7,27


<H2>Предобработка данных

In [5]:
# Просмотрим данные
print("Number of unique companies:", df.Company.unique().shape[0])  # 422 компаний

Number of unique companies: 422


In [6]:
print("Number of time periods per company:")
print(pd.crosstab(df.Company, df.Time.sum()))  # Некоторые из компаний имеют < 5 временных периода

Number of time periods per company:
col_0    27644
Company       
1            4
2           14
3            1
4           14
5           14
6           14
7           11
8           14
9           14
10          14
11           9
12           1
13           2
14           8
15           6
16           5
17           9
18           6
19           3
20          14
21          14
22          14
23          14
24          14
25          14
26          14
27           2
28          14
29          14
30          14
...        ...
393          3
394         14
395         11
396          1
397          5
398          2
399          5
400          3
401          5
402          6
403         13
404          3
405          7
406          2
407          3
408         14
409         13
410         12
411          4
412          3
413          1
414          2
415          7
416          9
417          4
418          2
419          3
420          3
421          6
422          8

[422 rows x 1 colu

Рассмотрим данные по группам на компанию

In [7]:
grouped_company = df.groupby('Company')

# Возьмем первые 5 групп
group_gen = ((name, group) for name, group in grouped_company)
for name, group in itertools.islice(group_gen, 5):
    # Для каждой группы
    print('-------------------------------------')
    print("Data of Company", name)
    print(group.head(15))

-------------------------------------
Data of Company 1
   Company  Time  Financial Distress      x1        x2       x3       x4  \
0        1     1            0.010636  1.2810  0.022934  0.87454  1.21640   
1        1     2           -0.455970  1.2700  0.006454  0.82067  1.00490   
2        1     3           -0.325390  1.0529 -0.059379  0.92242  0.72926   
3        1     4           -0.566570  1.1131 -0.015229  0.85888  0.80974   

         x5        x6       x7  ...      x74    x75     x76     x77   x78  \
0  0.060940  0.188270  0.52510  ...   85.437  27.07  26.102  16.000  16.0   
1 -0.014080  0.181040  0.62288  ...  107.090  31.31  30.194  17.000  16.0   
2  0.020476  0.044865  0.43292  ...  120.870  36.07  35.273  17.000  15.0   
3  0.076037  0.091033  0.67546  ...   54.806  39.80  38.377  17.167  16.0   

   x79  x80       x81  x82  x83  
0  0.2   22  0.060390   30   49  
1  0.4   22  0.010636   31   50  
2 -0.2   22 -0.455970   32   51  
3  5.6   22 -0.325390   33   52  

[4 row

<H3>Проблемы с простым разделением временных рядов::

Как можно увидеть на основе приведенного выше вывода кода, хотя у нас есть 1 общая временная переменная (Time), поскольку у нас несколько компаний, у нас может быть несколько строк, принадлежащих одному и тому же значению Time (например, 1 строка для Time 1 + Company 1 , другая строка для Time 1 + Company 2 и т. д.).

**Из этого возникают опредеденные ограничения, в частности:**
Это не позволяет нам использовать sklearn.model_selection.TimeSeriesSplit, поскольку эта функция предполагает, 
что каждая строка представляет точку данных из уникального момента времени
(и строки расположены в соответствии с возрастающим значением времени).


Мы все еще можем достичь нашей цели другим путем:
1) Разделить набор данных на несколько групп - по 1 группе на компанию. 

2) Для каждой группы вывести индексы для прямой цепочки.

3) Объединить список индексов для каждой группы в один окончательный список индексов

<H3>Работа с фиктивными данными

Как упоминалось в Словаре данных, одна из функций на самом деле является категориальной. 
Поэтому создадим фиктивные столбцы:

In [9]:
dummy_cols = pd.get_dummies(df[['x80']], prefix='dummy', columns=['x80'], drop_first=True)

print(dummy_cols.head())

   dummy_2  dummy_3  dummy_4  dummy_5  dummy_6  dummy_7  dummy_8  dummy_9  \
0        0        0        0        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   
2        0        0        0        0        0        0        0        0   
3        0        0        0        0        0        0        0        0   
4        0        0        0        0        0        0        0        0   

   dummy_10  dummy_11  ...  dummy_28  dummy_29  dummy_30  dummy_31  dummy_32  \
0         0         0  ...         0         0         0         0         0   
1         0         0  ...         0         0         0         0         0   
2         0         0  ...         0         0         0         0         0   
3         0         0  ...         0         0         0         0         0   
4         0         0  ...         0         1         0         0         0   

   dummy_33  dummy_34  dummy_35  dummy_36  dummy_37  
0 

In [10]:
x_cols = [col for col in df.columns if all([col.startswith('x'), col != 'x80'])]
df_transformed = pd.concat([df[['Company', 'Time', 'Financial Distress'] + x_cols].reset_index(drop=True),
                            pd.DataFrame(data=dummy_cols)], axis=1)

df_transformed.head()

,Company,Time,Financial Distress,x1,x2,x3,x4,x5,x6,x7,...,dummy_28,dummy_29,dummy_30,dummy_31,dummy_32,dummy_33,dummy_34,dummy_35,dummy_36,dummy_37
0,1,1,0.010636,1.2810,0.022934,0.87454,1.21640,0.060940,0.188270,0.52510,...,0,0,0,0,0,0,0,0,0,0
1,1,2,-0.455970,1.2700,0.006454,0.82067,1.00490,-0.014080,0.181040,0.62288,...,0,0,0,0,0,0,0,0,0,0
2,1,3,-0.325390,1.0529,-0.059379,0.92242,0.72926,0.020476,0.044865,0.43292,...,0,0,0,0,0,0,0,0,0,0
3,1,4,-0.566570,1.1131,-0.015229,0.85888,0.80974,0.076037,0.091033,0.67546,...,0,0,0,0,0,0,0,0,0,0
4,2,1,1.357300,1.0623,0.107020,0.81460,0.83593,0.199960,0.047800,0.74200,...,0,1,0,0,0,0,0,0,0,0


Покончив с описанным выше этапом предварительной обработки, мы перейдем ко второй части проектирования функций - созданию отстающих функций (опять же, отстающих функций для каждой группы):

In [11]:
def lagged_features(df_long, lag_features, window=2, lag_prefix='lag', lag_prefix_sep='_'):

    if not isinstance(lag_features, list):
        lag_features = [lag_features]

    if window <= 0:
        return df_long

    df_working = df_long[lag_features].copy()
    df_result = df_long.copy()
    for i in range(1, window+1):
        df_temp = df_working.shift(i)
        df_temp.columns = [lag_prefix + lag_prefix_sep + str(i) + lag_prefix_sep + x
                           for x in df_temp.columns]
        df_result = pd.concat([df_result.reset_index(drop=True),
                               df_temp.reset_index(drop=True)],
                               axis=1)

    return df_result


grouped_company = df_transformed.groupby('Company')
cols_to_lag = [col for col in df_transformed.columns if col.startswith('x')]
df_cross = pd.DataFrame()

for name, group in grouped_company:
    print('Working on group:', name, 'with shape', group.shape)
    df_cross = pd.concat([df_cross.reset_index(drop=True),
                          lagged_features(group, cols_to_lag).reset_index(drop=True)],
                         axis=0)
    print('Shape of df_cross', df_cross.shape)
    
df_cross = df_cross.dropna()
df_cross.head()

Working on group: 1 with shape (4, 121)
Shape of df_cross (4, 285)
Working on group: 2 with shape (14, 121)
Shape of df_cross (18, 285)
Working on group: 3 with shape (1, 121)
Shape of df_cross (19, 285)
Working on group: 4 with shape (14, 121)
Shape of df_cross (33, 285)
Working on group: 5 with shape (14, 121)
Shape of df_cross (47, 285)
Working on group: 6 with shape (14, 121)
Shape of df_cross (61, 285)
Working on group: 7 with shape (11, 121)
Shape of df_cross (72, 285)
Working on group: 8 with shape (14, 121)
Shape of df_cross (86, 285)
Working on group: 9 with shape (14, 121)
Shape of df_cross (100, 285)
Working on group: 10 with shape (14, 121)
Shape of df_cross (114, 285)
Working on group: 11 with shape (9, 121)
Shape of df_cross (123, 285)
Working on group: 12 with shape (1, 121)
Shape of df_cross (124, 285)
Working on group: 13 with shape (2, 121)
Shape of df_cross (126, 285)
Working on group: 14 with shape (8, 121)
Shape of df_cross (134, 285)
Working on group: 15 with shap

,Company,Time,Financial Distress,x1,x2,x3,x4,x5,x6,x7,...,lag_2_x73,lag_2_x74,lag_2_x75,lag_2_x76,lag_2_x77,lag_2_x78,lag_2_x79,lag_2_x81,lag_2_x82,lag_2_x83
2,1,3,-0.32539,1.05290,-0.059379,0.92242,0.72926,0.020476,0.044865,0.43292,...,36.0,85.437,27.07,26.102,16.0,16.0,0.2,0.060390,30.0,49.0
3,1,4,-0.56657,1.11310,-0.015229,0.85888,0.80974,0.076037,0.091033,0.67546,...,36.0,107.090,31.31,30.194,17.0,16.0,0.4,0.010636,31.0,50.0
6,2,3,1.20020,0.97059,0.076064,0.90677,0.80980,0.165920,-0.024649,0.73660,...,36.0,85.437,27.07,26.102,16.0,16.0,0.2,1.251000,7.0,27.0
7,2,4,2.23480,1.05900,0.130200,0.81811,0.87599,0.234450,0.045576,0.78727,...,36.0,107.090,31.31,30.194,17.0,16.0,0.4,1.357300,8.0,28.0
8,2,5,1.34050,1.12450,0.147840,0.75871,1.07990,0.276440,0.089408,0.80356,...,35.0,120.870,36.07,35.273,17.0,15.0,-0.2,0.007188,9.0,29.0


<H5>Разделение временных рядов на группу

Затем мы напишем вспомогательную функцию для создания разделений временных рядов для прямой цепочки. Функция вернет список кортежей. Каждый кортеж будет содержать 2 значения - индекс поезда и индекс теста.

In [12]:
# Create Time-Series sampling function to draw train-test splits
def ts_sample(df_input, train_rows, test_rows):
    if df_input.shape[0] <= train_rows:
        return [(df_input.index, pd.Index([]))]

    i = 0
    train_lower, train_upper = 0, train_rows + test_rows*i
    test_lower, test_upper = train_upper, min(train_upper + test_rows, df_input.shape[0])

    result_list = []
    while train_upper < df_input.shape[0]:
        # Get indexes into result_list
        result_list += [(df_input.index[train_lower:train_upper],
                         df_input.index[test_lower:test_upper])]

        # Update counter and calculate new indexes
        i += 1
        train_upper = train_rows + test_rows*i
        test_lower, test_upper = train_upper, min(train_upper + test_rows, df_input.shape[0])

    return result_list

Следующим шагом является использование ts_sample для каждой группы данных. Это приведет к созданию 1 списка индексных кортежей для каждой группы.

Более того, поскольку количество периодов времени в группе не одинаково, размер этих периодов также будет варьироваться. Следовательно, нам понадобится способ дополнить более короткие группы.

In [14]:
grouped_company_cross = df_cross.groupby('Company')
acc = []
max_size = 0
for name, group in grouped_company_cross:
    group_res = ts_sample(group, 4, 4)
    acc += [group_res]

    if len(group_res) > max_size:
        max_size = len(group_res)

        for idx, list_i in enumerate(acc):
            if len(list_i) < max_size:
                last_train, last_test = list_i[-1][0], list_i[-1][1]
                list_i[len(list_i):max_size] = [(last_train.union(last_test),
                                                 pd.Index([]))] * (max_size - len(list_i))

                acc[idx] = list_i

    elif len(group_res) < max_size:
        last_train, last_test = acc[-1][-1][0], acc[-1][-1][1]
        acc[-1] = acc[-1] + [(last_train.union(last_test), pd.Index([]))] * (max_size - len(acc[-1]))


print(acc[0:2])

[[(Int64Index([2, 3], dtype='int64'), Index([], dtype='object')), (Int64Index([2, 3], dtype='int64'), Index([], dtype='object'))], [(Int64Index([6, 7, 8, 9], dtype='int64'), Int64Index([10, 11, 12, 13], dtype='int64')), (Int64Index([6, 7, 8, 9, 10, 11, 12, 13], dtype='int64'), Int64Index([14, 15, 16, 17], dtype='int64'))]]


In [15]:
flat_acc = []
for idx, list_i in enumerate(acc):
    if len(flat_acc) == 0:
        flat_acc += list_i
        continue

    for inner_idx, tuple_i in enumerate(list_i):
        flat_acc[inner_idx] = (flat_acc[inner_idx][0].union(tuple_i[0]),
                               flat_acc[inner_idx][1].union(tuple_i[1]))


print(flat_acc[0:2])

[(Int64Index([   2,    3,    4,    5,    6,    7,    8,    9,   21,   22,
            ...
            3641, 3642, 3648, 3649, 3654, 3657, 3660, 3661, 3662, 3663],
           dtype='int64', length=1260), Int64Index([   6,    7,   10,   11,   12,   13,   25,   26,   27,   28,
            ...
            3603, 3604, 3614, 3615, 3616, 3617, 3636, 3643, 3644, 3645],
           dtype='int64', length=919)), (Int64Index([   2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
            ...
            3644, 3645, 3648, 3649, 3654, 3657, 3660, 3661, 3662, 3663],
           dtype='int64', length=2177), Int64Index([  14,   15,   16,   17,   29,   30,   31,   32,   43,   44,
            ...
            3565, 3591, 3592, 3593, 3594, 3605, 3606, 3607, 3618, 3619],
           dtype='int64', length=683))]


<H2>Моделирование

Столбец «Финансовое бедствие» содержит реальные значения, содержащие как положительные, так и отрицательные значения. 
Согласно словарю данных, мы должны рассматривать компанию в финансовом положении, если в столбце «Финансовый кризис» <= -0,50.
Соответственно, мы превратим эту проблему в проблему классификации, используя это определение

In [17]:
# преобразование Financial Distress column в 0 или 1
df_model = df_cross.copy()
df_model['Financial Distress'] = ['0' if x > -0.50 else '1' for x in df_model['Financial Distress'].values]

df_model.head()

,Company,Time,Financial Distress,x1,x2,x3,x4,x5,x6,x7,...,lag_2_x73,lag_2_x74,lag_2_x75,lag_2_x76,lag_2_x77,lag_2_x78,lag_2_x79,lag_2_x81,lag_2_x82,lag_2_x83
2,1,3,0,1.05290,-0.059379,0.92242,0.72926,0.020476,0.044865,0.43292,...,36.0,85.437,27.07,26.102,16.0,16.0,0.2,0.060390,30.0,49.0
3,1,4,1,1.11310,-0.015229,0.85888,0.80974,0.076037,0.091033,0.67546,...,36.0,107.090,31.31,30.194,17.0,16.0,0.4,0.010636,31.0,50.0
6,2,3,0,0.97059,0.076064,0.90677,0.80980,0.165920,-0.024649,0.73660,...,36.0,85.437,27.07,26.102,16.0,16.0,0.2,1.251000,7.0,27.0
7,2,4,0,1.05900,0.130200,0.81811,0.87599,0.234450,0.045576,0.78727,...,36.0,107.090,31.31,30.194,17.0,16.0,0.4,1.357300,8.0,28.0
8,2,5,0,1.12450,0.147840,0.75871,1.07990,0.276440,0.089408,0.80356,...,35.0,120.870,36.07,35.273,17.0,15.0,-0.2,0.007188,9.0,29.0


In [18]:
# For each entry in flat_acc, perform train and test and plot metrics
dependent_cols = [col for col in df_model.columns if col != 'Financial Distress']
independent_col = ['Financial Distress']
for idx, tuple_i in enumerate(flat_acc):
    print('---------------------------------------')
    X_train, X_test = df_model.loc[tuple_i[0]][dependent_cols], df_model.loc[tuple_i[1]][dependent_cols]
    y_train, y_test = df_model.loc[tuple_i[0]][independent_col], df_model.loc[tuple_i[1]][independent_col]
    
    # Fit logistic regression model to train data and test on test data
    lr_mod = LogisticRegression(C=0.01, penalty='l2')  # These should be determined by nested cv
    lr_mod.fit(X_train, y_train)
    
    y_pred_proba = lr_mod.predict_proba(X_test)
    y_pred = lr_mod.predict(X_test)
    
    # Print Confusion Matrix and ROC AUC score
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred))
    
    print('ROC AUC score:')
    print(roc_auc_score(y_test['Financial Distress'].astype(int), y_pred_proba[:, 1]))

---------------------------------------


C:\Users\darya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\darya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\darya\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\darya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\darya\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y wa

Confusion Matrix:
[[880   2]
 [ 39   0]]
ROC AUC score:
0.8008314436885865
---------------------------------------
Confusion Matrix:
[[659   1]
 [ 23   0]]
ROC AUC score:
0.7546772068511198


C:\Users\darya\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


<H2>Вывод

Эта модель упускает многие тонкости данных на которые следует обратить внимание

1) **Работа с несбалансированными данными:**

Обработка искаженных данных включает в себя некоторую разновидность недостаточной выборки класса большинства + передискретизации класса меньшинства.

(В этой модели это пропущено, но это очень важно для получения хорошего классификатора.)

2) **Вложенная проверка для выбора гиперпараметров (например, значения C в логистической регрессии):**

На практике мы бы использовали вложенную перекрестную проверку, чтобы также определить гиперпараметры, которые мы здесь жестко запрограммировали.